# eGov APIを利用して法令を抽出する


## 参考文献

- https://qiita.com/Lisphilar/items/39ad23ac7ade21313911
- https://laws.e-gov.go.jp/api/2/swagger-ui


In [ ]:
import re
import xml.etree.ElementTree as ET
from functools import lru_cache
from pprint import pprint
from xml.etree import ElementTree

# pip install requests
import requests

In [ ]:
@lru_cache
def get_lawid_from_lawtitle(law_title: str) -> str:
    """APIから法令タイトルでヒットする法令IDを取得(完全一致のみ)"""
    url = f"https://laws.e-gov.go.jp/api/2/laws"
    r = requests.get(url, params={"response_format": "xml", "law_title": law_title})
    # XMLデータの解析
    root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))

    laws_elem = root.find("laws")
    if laws_elem is None:
        print("Error: 'laws' element not found in response.")
        return {}

    counter = 0
    law_dict = {}  # 辞書{名称: 法令番号}の作成
    for law in laws_elem.findall("law"):  # loop over <law> elements
        counter += 1

        law_info = law.find("law_info")
        revision_info = law.find("revision_info")

        if law_info is None or revision_info is None:
            continue  # skip incomplete entries

        law_id: str = law_info.findtext("law_id", default="(no id)")
        law_num: str = law_info.findtext("law_num", default="(no number)")
        lawtitle: str = revision_info.findtext("law_title", default="(no title)")

        print(f"ID: {law_id}, Num: {law_num}, Title: {lawtitle}")
        law_dict[lawtitle] = law_id
    print(f"Number of laws: {counter}")
    return law_dict[law_title]  # allow exact match

In [ ]:
get_lawid_from_lawtitle("大気汚染防止法")

ID: 343AC0000000097, Num: 昭和四十三年法律第九十七号, Title: 大気汚染防止法
ID: 343CO0000000329, Num: 昭和四十三年政令第三百二十九号, Title: 大気汚染防止法施行令
ID: 343M50000800058, Num: 昭和四十三年運輸省令第五十八号, Title: 大気汚染防止法第二条第十七項の自動車及び原動機付自転車を定める省令
ID: 346M50000102002, Num: 昭和四十六年総理府・厚生省令第二号, Title: 大気汚染防止法第二十一条第一項の規定に基づく自動車排出ガスによる大気の汚染の限度を定める省令
ID: 346M50000500001, Num: 昭和四十六年厚生省・通商産業省令第一号, Title: 大気汚染防止法施行規則
Number of laws: 5


'343AC0000000097'

In [ ]:
from typing import Literal


def get_lawdata_from_law_id(law_id: str, type: Literal["xml", "list"]):
    """法令IDから法令データを取得"""
    url = f"https://laws.e-gov.go.jp/api/2/law_data/{law_id}"
    r = requests.get(url, params={"response_format": "xml"})
    if r.status_code != 200:
        print(f"Error fetching law data for ID {law_id}: {r.status_code}")
        return None
    if type == "xml":
        return r.content.decode(encoding="utf-8")

    if type == "list":
        # XMLデータの解析
        root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))
        contents = [e.text.strip() for e in root.iter() if e.text]
        return [t for t in contents if t]


def save_xml_string_to_file(xml_string: str, filename: str):
    """save xml string to txt"""
    with open(filename, "w", encoding="utf-8") as f:
        f.write(xml_string)

In [ ]:
xml_text: str = get_lawdata_from_law_id("343AC0000000097", "xml")

In [ ]:
xml_text

'<law_data_response><attached_files_info/><law_info><law_type>Act</law_type><law_id>343AC0000000097</law_id><law_num>昭和四十三年法律第九十七号</law_num><law_num_era>Showa</law_num_era><law_num_year>43</law_num_year><law_num_type>Act</law_num_type><law_num_num>097</law_num_num><promulgation_date>1968-06-10</promulgation_date></law_info><revision_info><law_revision_id>343AC0000000097_20250601_504AC0000000068</law_revision_id><law_type>Act</law_type><law_title>大気汚染防止法</law_title><law_title_kana>たいきおせんぼうしほう</law_title_kana><abbrev>大防法</abbrev><category>環境保全</category><updated>2024-07-22T11:28:34+09:00</updated><amendment_promulgate_date>2022-06-17</amendment_promulgate_date><amendment_enforcement_date>2025-06-01</amendment_enforcement_date><amendment_enforcement_comment/><amendment_scheduled_enforcement_date/><amendment_law_id>504AC0000000068</amendment_law_id><amendment_law_title>刑法等の一部を改正する法律の施行に伴う関係法律の整理等に関する法律</amendment_law_title><amendment_law_title_kana/><amendment_law_num>令和四年法律第六十八号</amendmen

In [ ]:
save_xml_string_to_file(xml_text, "law.xml")

In [ ]:
def extract_main_sections_from_xml(xml_file_path):
    """TOC, MainProvision,SupplProvisionの3つを取得"""
    #  tree = ET.parse(xml_file_path)
    #  root = tree.getroot()
    root = ET.fromstring(toc_xml)

    # law_infoタグを取得
    law_full_text = root.find("law_full_text")
    if law_full_text is None:
        raise ValueError("law_full_textタグが見つかりません")

    # <Law> の中にある <LawBody> を探す
    law = law_full_text.find("Law")
    if law is None:
        raise ValueError("<Law> タグが <law_full_text> 内に見つかりません")

    law_body = law.find("LawBody")
    if law_body is None:
        raise ValueError("<LawBody> タグが <Law> 内に見つかりません")

    # 対象の3つのタグを取得
    toc = law_body.find("TOC")
    main_prov = law_body.find("MainProvision")
    suppl_provs = law_body.findall("SupplProvision")

    return {
        "TOC": ET.tostring(toc, encoding="unicode") if toc is not None else None,
        "MainProvision": (
            ET.tostring(main_prov, encoding="unicode")
            if main_prov is not None
            else None
        ),
        # "SupplProvision": (
        #     ET.tostring(suppl_prov, encoding="unicode")
        #     if suppl_prov is not None
        #     else None
        # ),
        "SupplProvision": (
            [ET.tostring(s, encoding="unicode") for s in suppl_provs]
            if suppl_provs
            else None
        ),
    }


# 使用例
sections = extract_main_sections_from_xml("xml_text")

print("=== TOC ===")
print(sections["TOC"] or "なし")

print("\n=== MainProvision ===")
print(sections["MainProvision"] or "なし")

print("\n=== SupplProvision ===")
print(sections["SupplProvision"] or "なし")

NameError: name 'toc_xml' is not defined

In [ ]:
import xml.etree.ElementTree as ET


def clean_toc(toc_xml: str) -> str:
    # XML文字列をパース
    toc_elem = ET.fromstring(toc_xml)

    # 出力用リスト
    lines = []

    # TOCLabel を追加
    label = toc_elem.find("TOCLabel")
    if label is not None and label.text:
        lines.append(label.text.strip())

    # TOCChapter を順に処理
    for chapter in toc_elem.findall("TOCChapter"):
        title = chapter.find("ChapterTitle")
        article_range = chapter.find("ArticleRange")
        if title is not None and article_range is not None:
            line = f"{title.text.strip()}{article_range.text.strip()}"
            lines.append(line)

    # TOCSupplProvision のラベル（附則）を追加
    suppl = toc_elem.find("TOCSupplProvision")
    if suppl is not None:
        suppl_label = suppl.find("SupplProvisionLabel")
        if suppl_label is not None and suppl_label.text:
            lines.append(suppl_label.text.strip())

    return "\n".join(lines)

In [ ]:
clean_toc(sections["TOC"])

'目次\n第一章\u3000総則（第一条・第二条）\n第二章\u3000ばい煙の排出の規制等（第三条―第十七条の二）\n第二章の二\u3000揮発性有機化合物の排出の規制等（第十七条の三―第十七条の十五）\n第二章の三\u3000粉じんに関する規制（第十八条―第十八条の二十五）\n第二章の四\u3000水銀等の排出の規制等（第十八条の二十六―第十八条の四十）\n第二章の五\u3000有害大気汚染物質対策の推進（第十八条の四十一―第十八条の四十五）\n第三章\u3000自動車排出ガスに係る許容限度等（第十九条―第二十一条の二）\n第四章\u3000大気の汚染の状況の監視等（第二十二条―第二十四条）\n第四章の二\u3000損害賠償（第二十五条―第二十五条の六）\n第五章\u3000雑則（第二十六条―第三十二条）\n第六章\u3000罰則（第三十三条―第三十七条）\n附則'

In [ ]:
def extract_text_from_xml(xml: str):
    """MainProvisionを処理する"""

    def get_ruby_text(element):
        # <Ruby>漢字<Rt>読み</Rt></Ruby> → 漢字（読み）
        base = "".join(element.itertext())
        rt = element.find("Rt")
        if rt is not None:
            return f"{element.text}（{rt.text}）"
        return base

    root = ET.fromstring(xml)

    lines = []

    for chapter in root.findall("Chapter"):
        chapter_title = chapter.findtext("ChapterTitle")
        if chapter_title:
            lines.append(chapter_title.strip())
            lines.append("")  # 改行

        for article in chapter.findall("Article"):
            caption = article.findtext("ArticleCaption")
            title = article.findtext("ArticleTitle")

            if caption:
                lines.append(caption.strip())
            if title:
                lines.append(title.strip())

            for para in article.findall("Paragraph"):
                para_num = para.findtext("ParagraphNum")
                if para_num:
                    lines.append(para_num.strip())

                # 本文の文を追加
                para_sentence = para.find("ParagraphSentence")
                if para_sentence is not None:
                    for sentence in para_sentence.findall(".//Sentence"):
                        sentence_text = ""
                        for elem in sentence.iter():
                            if elem.tag == "Ruby":
                                sentence_text += get_ruby_text(elem)
                            elif elem.text:
                                sentence_text += elem.text
                        lines.append(sentence_text.strip())

                # 項目（Item）も処理
                for item in para.findall("Item"):
                    item_title = item.findtext("ItemTitle")
                    if item_title:
                        lines.append(item_title.strip())

                    item_sentence = item.find("ItemSentence")
                    if item_sentence is not None:
                        for sentence in item_sentence.findall(".//Sentence"):
                            sentence_text = ""
                            for elem in sentence.iter():
                                if elem.tag == "Ruby":
                                    sentence_text += get_ruby_text(elem)
                                elif elem.text:
                                    sentence_text += elem.text
                            lines.append(sentence_text.strip())

    return "\n".join(lines)

In [ ]:
text = extract_text_from_xml(sections["MainProvision"])

with open("mainprovision.txt", "w", encoding="utf-8") as f:
    f.write(text)

text

'第一章\u3000総則\n\n（目的）\n第一条\nこの法律は、工場及び事業場における事業活動並びに建築物等の解体等に伴うばい煙、揮発性有機化合物及び粉じんの排出等を規制し、水銀に関する水俣条約（以下「条約」という。）の的確かつ円滑な実施を確保するため工場及び事業場における事業活動に伴う水銀等の排出を規制し、有害大気汚染物質対策の実施を推進し、並びに自動車排出ガスに係る許容限度を定めること等により、大気の汚染に関し、国民の健康を保護するとともに生活環境を保全し、並びに大気の汚染に関して人の健康に係る被害が生じた場合における事業者の損害賠償の責任について定めることにより、被害者の保護を図ることを目的とする。\n（定義等）\n第二条\nこの法律において「ばい煙」とは、次の各号に掲げる物質をいう。\n一\n燃料その他の物の燃焼に伴い発生するいおう酸化物\n二\n燃料その他の物の燃焼又は熱源としての電気の使用に伴い発生するばいじん\n三\n物の燃焼、合成、分解その他の処理（機械的処理を除く。）に伴い発生する物質のうち、カドミウム、塩素、弗（ふつ）ふつ\n２\nこの法律において「ばい煙発生施設」とは、工場又は事業場に設置される施設でばい煙を発生し、及び排出するもののうち、その施設から排出されるばい煙が大気の汚染の原因となるもので政令で定めるものをいう。\n３\nこの法律において「ばい煙処理施設」とは、ばい煙発生施設において発生するばい煙を処理するための施設及びこれに附属する施設をいう。\n４\nこの法律において「揮発性有機化合物」とは、大気中に排出され、又は飛散した時に気体である有機化合物（浮遊粒子状物質及びオキシダントの生成の原因とならない物質として政令で定める物質を除く。）をいう。\n５\nこの法律において「揮発性有機化合物排出施設」とは、工場又は事業場に設置される施設で揮発性有機化合物を排出するもののうち、その施設から排出される揮発性有機化合物が大気の汚染の原因となるものであつて、揮発性有機化合物の排出量が多いためにその規制を行うことが特に必要なものとして政令で定めるものをいう。\n６\n前項の政令は、事業者が自主的に行う揮発性有機化合物の排出及び飛散の抑制のための取組が促進されるよう十分配慮して定めるものとする。\n７\nこの法律において「粉じん」とは、物の破砕、選

In [ ]:
def clean_suppl_provision(xml_string):
    root = ET.fromstring(xml_string)
    output = []

    for para in root.findall(".//Paragraph"):
        # 見出し（段落キャプション）があれば取得
        caption = para.findtext("ParagraphCaption")
        if caption:
            output.append(f"（{caption.strip('（）')}）")

        # 段落番号
        para_num = para.findtext("ParagraphNum")
        line = f"{para_num}　" if para_num else ""

        # センテンスをすべて連結
        sentences = para.findall(".//Sentence")
        sentence_texts = [s.text.strip() for s in sentences if s.text]
        line += (
            "。".join(s.strip("。") for s in sentence_texts) + "。"
            if sentence_texts
            else ""
        )

        # 出力に追加
        output.append(line)

    return "\n".join(output)

In [ ]:
text = clean_suppl_provision(sections["SupplProvision"][0])

with open("supplprovision.txt", "w", encoding="utf-8") as f:
    f.write(text)

pprint(text)

('（施行期日）\n'
 '１\u3000'
 'この法律は、公布の日から起算して六月をこえない範囲内において政令で定める日から施行する。ただし、第四条第四項の規定は、公布の日から施行する。\n'
 '（ばい煙の排出の規制等に関する法律の廃止）\n'
 '２\u3000ばい煙の排出の規制等に関する法律（昭和三十七年法律第百四十六号。以下「旧法」という。）は、廃止する。\n'
 '（経過措置）\n'
 '３\u3000'
 'この法律の施行の際現に旧法第十二条の規定による実施の制限を受けている者についての第十条及び第十一条の規定の適用については、第十条中「その届出を受理した日」とあるのは「旧ばい煙の排出の規制等に関する法律第八条第一項又は第十条第一項の規定による届出を受理した日」と、第十一条第一項中「その届出が受理された日」とあるのは「旧ばい煙の排出の規制等に関する法律第八条第一項又は第十条第一項の規定による届出が受理された日」とする。\n'
 '４\u3000'
 'この法律の施行の際現に旧法第十六条第三項の規定により同条第一項又は第二項の規定を適用しないものとされているばい煙発生施設についての第十四条第三項の規定の適用については、同項中「同項に規定する指定地域となつた日又は同項に規定するばい煙発生施設となつた日」とあるのは「旧ばい煙の排出の規制等に関する法律第九条第一項に規定する指定地域となつた日又は同項に規定するばい煙発生施設となつた日」とする。\n'
 '５\u3000'
 'この法律の施行前に旧法第九条第一項の規定による届出をした者であつて、その届出をした日からこの法律の施行の日までの期間が六十日に満たないものの当該届出に係るばい煙発生施設についての第十四条第三項ただし書の規定の適用については、同項ただし書中「当該届出が受理された日」とあるのは、「旧ばい煙の排出の規制等に関する法律第十条第一項の規定による届出をした日」とする。\n'
 '６\u3000'
 'この法律の施行の際現に旧法第二十三条第一項の規定によつて委嘱されている仲介員候補者又は同法第二十四条第一項の規定によつて指定されている仲介員は、それぞれ、第二十三条第一項の規定によつて委嘱され、又は第二十四条第一項の規定によつて指定されたものとみなす。\n'
 '７\u3000'
 '前項に規定する場合

In [ ]:
import xml.etree.ElementTree as ET


def extract_text(elem, indent=0):
    """要素とその子要素からテキストを整形して抽出"""
    lines = []
    tag = elem.tag
    text = (elem.text or "").strip()

    if tag == "LawTitle":
        lines.append(f"【法令名】{text}")
    elif tag == "EnactStatement":
        lines.append(f"【制定文】{text}")
    elif tag == "Preamble":
        lines.append(f"【前文】{text}")
    elif tag == "Article":
        article_num = elem.attrib.get("Num", "")
        lines.append(" " * indent + f"第{article_num}条 {text}")
    elif tag == "Paragraph":
        para_num = elem.attrib.get("Num", "")
        prefix = f"{para_num}　" if para_num else ""
        lines.append(" " * indent + prefix + text)
    elif tag == "Item":
        item_num = elem.attrib.get("Num", "")
        lines.append(" " * indent + f"{item_num}) {text}")
    elif tag == "Subitem1":
        lines.append(" " * indent + f"・{text}")
    elif tag in ["Chapter", "Section", "Subsection", "Division", "Part"]:
        caption = elem.findtext("Caption", default="").strip()
        num = elem.attrib.get("Num", "")
        lines.append(" " * indent + f"【{caption or tag} {num}】")
    else:
        if text:
            lines.append(" " * indent + text)

    # 再帰的に子要素も抽出
    for child in elem:
        lines.extend(extract_text(child, indent + 2))

    return lines


def parse_law_from_string(xml_string):
    """XML文字列から法令テキストを抽出"""
    root = ET.fromstring(xml_string)

    # law_data_response > law_info を探す
    law_info = root.find(".//law_info")
    print("law_info", law_info)
    if law_info is None:
        raise ValueError("law_info 要素が見つかりません")

    output_lines = []

    for tag in [
        "LawTitle",
        "EnactStatement",
        "Preamble",
        "MainProvision",
        "SupplProvision",
    ]:
        for elem in law_info.findall(f".//{tag}"):
            output_lines.extend(extract_text(elem))

    return "\n".join(output_lines)


# if __name__ == "__main__":
#     xml_file = "your_law_file.xml"  # XMLファイルパス
#     law_text = parse_law_xml(xml_file)
#     print(law_text)

In [ ]:
law_text = parse_law_from_string(xml_text)
print(law_text)

law_info <Element 'law_info' at 0x11522fe20>



In [ ]:
def test(xml_text):
    # パース
    root = ElementTree.fromstring(xml_text)

    # <law_full_text><Law> に到達
    law_elem = root.find(".//law_full_text/Law")
    if law_elem is None:
        return "法令本文が見つかりません"

    # タイトル抽出
    law_title = law_elem.findtext(".//LawTitle")
    law_num = law_elem.findtext(".//LawNum")
    output = [f"{law_title}（{law_num}）", ""]

    # 目次抽出
    toc_elem = law_elem.find(".//TOC")
    if toc_elem is not None:
        output.append("【目次】")
        for chapter in toc_elem.findall("TOCChapter"):
            chapter_title = chapter.findtext("ChapterTitle")
            article_range = chapter.findtext("ArticleRange")
            if chapter_title:
                line = f"{chapter_title}"
                if article_range:
                    line += f"　{article_range}"
                output.append(line)
    else:
        output.append("目次情報が見つかりません")

    return "\n".join(output)

In [ ]:
text = get_lawdata_from_law_id("343AC0000000097", type="xml")
text = text.replace("\u3000", "")
text

'<law_data_response><attached_files_info/><law_info><law_type>Act</law_type><law_id>343AC0000000097</law_id><law_num>昭和四十三年法律第九十七号</law_num><law_num_era>Showa</law_num_era><law_num_year>43</law_num_year><law_num_type>Act</law_num_type><law_num_num>097</law_num_num><promulgation_date>1968-06-10</promulgation_date></law_info><revision_info><law_revision_id>343AC0000000097_20250601_504AC0000000068</law_revision_id><law_type>Act</law_type><law_title>大気汚染防止法</law_title><law_title_kana>たいきおせんぼうしほう</law_title_kana><abbrev>大防法</abbrev><category>環境保全</category><updated>2024-07-22T11:28:34+09:00</updated><amendment_promulgate_date>2022-06-17</amendment_promulgate_date><amendment_enforcement_date>2025-06-01</amendment_enforcement_date><amendment_enforcement_comment/><amendment_scheduled_enforcement_date/><amendment_law_id>504AC0000000068</amendment_law_id><amendment_law_title>刑法等の一部を改正する法律の施行に伴う関係法律の整理等に関する法律</amendment_law_title><amendment_law_title_kana/><amendment_law_num>令和四年法律第六十八号</amendmen

In [ ]:
get_lawdata_from_law_id("343AC0000000097", type="list")

['Act',
 '343AC0000000097',
 '昭和四十三年法律第九十七号',
 'Showa',
 '43',
 'Act',
 '097',
 '1968-06-10',
 '343AC0000000097_20250601_504AC0000000068',
 'Act',
 '大気汚染防止法',
 'たいきおせんぼうしほう',
 '大防法',
 '環境保全',
 '2024-07-22T11:28:34+09:00',
 '2022-06-17',
 '2025-06-01',
 '504AC0000000068',
 '刑法等の一部を改正する法律の施行に伴う関係法律の整理等に関する法律',
 '令和四年法律第六十八号',
 '3',
 'None',
 'false',
 'New',
 'CurrentEnforced',
 '昭和四十三年法律第九十七号',
 '大気汚染防止法',
 '目次',
 '第一章\u3000総則',
 '（第一条・第二条）',
 '第二章\u3000ばい煙の排出の規制等',
 '（第三条―第十七条の二）',
 '第二章の二\u3000揮発性有機化合物の排出の規制等',
 '（第十七条の三―第十七条の十五）',
 '第二章の三\u3000粉じんに関する規制',
 '（第十八条―第十八条の二十五）',
 '第二章の四\u3000水銀等の排出の規制等',
 '（第十八条の二十六―第十八条の四十）',
 '第二章の五\u3000有害大気汚染物質対策の推進',
 '（第十八条の四十一―第十八条の四十五）',
 '第三章\u3000自動車排出ガスに係る許容限度等',
 '（第十九条―第二十一条の二）',
 '第四章\u3000大気の汚染の状況の監視等',
 '（第二十二条―第二十四条）',
 '第四章の二\u3000損害賠償',
 '（第二十五条―第二十五条の六）',
 '第五章\u3000雑則',
 '（第二十六条―第三十二条）',
 '第六章\u3000罰則',
 '（第三十三条―第三十七条）',
 '附則',
 '第一章\u3000総則',
 '（目的）',
 '第一条',
 'この法律は、工場及び事業場における事業活動並びに建築物等の解体等に伴うばい煙、揮発性有機化合物及び粉じんの排出等を規制し、水銀に関する水俣条約（

In [ ]:
from typing import Literal


def get_lawdata_from_law_id_apiv1(law_id: str, type: Literal["xml", "list"]):
    """法令IDから法令データを取得"""
    url = f"https://laws.e-gov.go.jp/api/2/law_data/{law_id}"
    r = requests.get(url, params={"response_format": "xml"})
    if r.status_code != 200:
        print(f"Error fetching law data for ID {law_id}: {r.status_code}")
        return None
    if type == "xml":
        return r.content.decode(encoding="utf-8")

    if type == "list":
        # XMLデータの解析
        root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))
        contents = [e.text.strip() for e in root.iter() if e.text]
        return [t for t in contents if t]

In [ ]:
from xml.etree.ElementTree import Element


def parse_element(xml: Element, elem: str):
    elem_from_xml = xml.find(elem)
    if elem_from_xml is None:
        print(f"Error: '{elem}' element not found.")
        return None
    return elem_from_xml


def get_lawdata_from_law_id(law_id: str):
    """法令IDから法令データを取得"""
    url = f"https://laws.e-gov.go.jp/api/2/law_data/{law_id}"
    r = requests.get(url, params={"response_format": "xml"})
    if r.status_code != 200:
        print(f"Error fetching law data for ID {law_id}: {r.status_code}")
        return None
    print(r.content.decode(encoding="utf-8"))

    # XMLデータの解析
    root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))

    # extract law_full_text
    law_full_text_elem = root.find("law_full_text")
    if law_full_text_elem is None:
        print(f"Error: 'law_full_text' element not found for ID {law_id}.")
        return None
    law = law_full_text_elem.find("Law")
    if law is None:
        print(f"Error: 'Law' element not found in 'law_full_text' for ID {law_id}.")
        return None
    lawbody = law.find("LawBody")
    if lawbody is None:
        print(f"Error: 'LawBody' element not found in 'Law' for ID {law_id}.")
        return None
    law_title = lawbody.findtext("LawTitle", default="(no title)")
    # 条文
    law_articles = lawbody.find("MainProvision")
    if law_articles is None:
        print(f"Error: 'MainProvision' element not found in 'LawBody' for ID {law_id}.")
        return None
    # 条文の各<Paragraph>要素を取得
    print(law_articles.tag, law_articles.attrib, law_articles.text)
    for law_article in law_articles:
        if law_article.tag == "Chapter":
            print(law_article.tag, law_article.attrib)
            # read Article
            for article in law_article:
                article_title = article.findtext("ArticleTitle", default="(no title)")
                print(f"  {article.tag} {article.attrib} {article_title}")
                if article.tag == "Article":
                    print(article.tag, article.attrib)
                    for paragraph in article:
                        if paragraph.tag == "Paragraph":
                            print(paragraph.tag, paragraph.attrib, paragraph.text)
                            for sentence in paragraph:
                                if sentence.tag == "ParagraphSentence":
                                    print(sentence.tag, sentence.attrib, sentence.text)
                                    for sentence_elem in sentence:
                                        if sentence_elem.tag == "Sentence":
                                            print(
                                                sentence_elem.tag,
                                                sentence_elem.attrib,
                                                sentence_elem.text,
                                            )
                                            # ここで必要な処理を行う
                                            # 例: print(sentence_elem.text)
                                            # もしくは、findtextを使って特定のテキストを取得
                                            print(sentence_elem.findtext("Sentence"))
                                    # print(sentence.findtext("Sentence"))

    print(f"law_title = {law_title}")
    print(f"law = {law}")
    return law_full_text_elem

In [ ]:
get_lawdata_from_law_id("343AC0000000097")

In [ ]:
@lru_cache
def get_law_dict():
    """APIから各法令種別に含まれる法令リストを取得"""
    url = f"https://laws.e-gov.go.jp/api/2/laws"
    r = requests.get(url, params={"response_format": "xml", "limit": 10000})
    # XMLデータの解析
    root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))
    print(r.content.decode(encoding="utf-8"))
    print(root)
    # print(root.getroot())
    print(f"root.tag {root.tag}")
    print(f"root.attrib {root.attrib}")
    print(f"root.find('laws') = {root.find('laws')}")

    laws_elem = root.find("laws")
    if laws_elem is None:
        print("Error: 'laws' element not found in response.")
        return {}

    counter = 0
    for law in laws_elem.findall("law"):  # loop over <law> elements
        counter += 1
        for law_info in law[0]:  # loop over law_info elements
            # print(law_type[0].tag, law_type[1].attrib, law_type[1].text)
            if law_info.tag == "law_num":
                print(law_info.tag, law_info.attrib, law_info.text)
            if law_info.tag == "law_id":
                print(law_info.tag, law_info.attrib, law_info.text)
        for revision_info in law[1]:
            if revision_info.tag == "law_title":
                print(revision_info.tag, revision_info.attrib, revision_info.text)
        # print(law.find("law_id"), law.find("law_num"))
    # for index, e in enumerate(root.iter()):
    #     print(f" ======== Element {index} =====")
    #     print(e.tag, e.text, e.attrib)
    print(f"Number of laws: {counter}")
    # 辞書{名称: 法令番号}の作成
    names = [e.text for e in root.iter() if e.tag == "LawName"]
    numbers = [e.text for e in root.iter() if e.tag == "LawNo"]
    return {name: num for (name, num) in zip(names, numbers)}

In [ ]:
pprint(get_law_dict(), compact=True)

In [ ]:
def get_law_number(keyword, category=1):
    """
    Return the law number.
    This will be retrieved from e-Gov (https://www.e-gov.go.jp/)

    Args:
        keyword (str): keyword of the law name
        category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

    Returns:
        dict(str, str): dictionary of law name (key) and law number (value)
    """
    law_dict = get_law_dict(category=category)
    return {k: v for (k, v) in law_dict.items() if keyword in k}

In [ ]:
print(get_law_number("医薬品の臨床試験", category=4))

{'医薬品の臨床試験の実施の基準に関する省令': '平成九年厚生省令第二十八号', '動物用医薬品の臨床試験の実施の基準に関する省令': '平成九年農林水産省令第七十五号'}


In [ ]:
@lru_cache
def get_raw(number):
    """
    Retrieve contents of the law specified with law number from e-Gov API.

    Args:
        number (str): Number of the law, like '平成九年厚生省令第二十八号'

    Returns:
        raw (list[str]): raw contents of J-GCP
    """
    url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{number}"
    r = requests.get(url)
    root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))
    contents = [e.text.strip() for e in root.iter() if e.text]
    return [t for t in contents if t]

In [ ]:
gcp_raw = get_raw("平成九年厚生省令第二十八号")
pprint(gcp_raw, compact=False)

['0',
 '平成九年厚生省令第二十八号',
 '平成九年厚生省令第二十八号',
 '医薬品の臨床試験の実施の基準に関する省令',
 '薬事法（昭和三十五年法律第百四十五号）第十四条第三項（同条第六項、同法第十九条の二第四項及び第二十三条において準用する場合を含む。）、第十四条の四第四項並びに第十四条の五第四項（これらの規定を同法第十九条の四及び第二十三条において準用する場合を含む。）、第八十条の二第一項、第四項及び第五項並びに第八十二条の規定に基づき、医薬品の臨床試験の実施の基準に関する省令を次のように定める。',
 '目次',
 '第一章\u3000総則',
 '（第一条―第三条）',
 '第二章\u3000治験の準備に関する基準',
 '第一節\u3000治験の依頼をしようとする者による治験の準備に関する基準',
 '（第四条―第十五条）',
 '第二節\u3000自ら治験を実施しようとする者による治験の準備に関する基準',
 '（第十五条の二―第十五条の九）',
 '第三章\u3000治験の管理に関する基準',
 '第一節\u3000治験依頼者による治験の管理に関する基準',
 '（第十六条―第二十六条）',
 '第二節\u3000自ら治験を実施する者による治験の管理に関する基準',
 '（第二十六条の二―第二十六条の十二）',
 '第四章\u3000治験を行う基準',
 '（第二十七条―第五十五条）',
 '第一節\u3000治験審査委員会',
 '（第二十七条―第三十四条）',
 '第二節\u3000実施医療機関',
 '（第三十五条―第四十一条）',
 '第三節\u3000治験責任医師',
 '（第四十二条―第四十九条）',
 '第四節\u3000被験者の同意',
 '（第五十条―第五十五条）',
 '第五章\u3000再審査等の資料の基準',
 '（第五十六条）',
 '第六章\u3000治験の依頼等の基準',
 '（第五十七条―第五十九条）',
 '附則',
 '第一章\u3000総則',
 '（趣旨）',
 '第一条',
 'この省令は、被験者の人権の保護、安全の保持及び福祉の向上を図り、治験の科学的な質及び成績の信頼性を確保するため、医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律（昭和三十五年法律第百四十五号。以下「法」という

In [ ]:
def preprocess_gcp(raw):
    """
    Perform pre-processing on raw contents of J-GCP.

    Args:
        raw (list[str]): raw contents of J-GCP

    Returns:
        str: pre-processed string of J-GCP

    Notes:
        - Article 56 will be removed.
        - Strings enclosed with （ and ） will be removed.
        - 「 and 」 will be removed.
    """
    # contents = raw[:]
    # Remove article 56
    contents = raw[: raw.index("第五十六条")]
    # Select sentenses
    contents = [s for s in contents if s.endswith("。")]
    # Join the sentenses
    gcp = "".join(contents)
    # 「 and 」 will be removed
    gcp = gcp.translate(str.maketrans({"「": "", "」": ""}))
    # 　Strings enclosed with （ and ） will be removed
    return re.sub("（[^（|^）]*）", "", gcp)

In [ ]:
gcp = preprocess_gcp(gcp_raw)
gcp

'薬事法第十四条第三項、第十四条の四第四項並びに第十四条の五第四項、第八十条の二第一項、第四項及び第五項並びに第八十二条の規定に基づき、医薬品の臨床試験の実施の基準に関する省令を次のように定める。この省令は、被験者の人権の保護、安全の保持及び福祉の向上を図り、治験の科学的な質及び成績の信頼性を確保するため、医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律第十四条第三項及び第十二項（同条第十五項及び法第十九条の二第五項において準用する場合並びに法第十四条の二の二第五項において読み替えて適用する場合を含む。以下同じ。）並びに法第十四条の四第五項及び第十四条の六第四項の厚生労働省令で定める基準のうち医薬品の臨床試験の実施に係るもの並びに法第八十条の二第一項、第四項及び第五項に規定する厚生労働省令で定める基準を定めるものとする。この省令において製造販売後臨床試験とは、医薬品の製造販売後の調査及び試験の実施の基準に関する省令第二条第一項第三号に規定する製造販売後臨床試験をいう。この省令において実施医療機関とは、治験又は製造販売後臨床試験を行う医療機関をいう。この省令において治験責任医師とは、実施医療機関において治験に係る業務を統括する医師又は歯科医師をいう。この省令において製造販売後臨床試験責任医師とは、実施医療機関において製造販売後臨床試験に係る業務を統括する医師又は歯科医師をいう。この省令において被験薬とは、治験の対象とされる薬物又は製造販売後臨床試験の対象とされる医薬品をいう。この省令において対照薬とは、治験又は製造販売後臨床試験において被験薬と比較する目的で用いられる薬物をいう。この省令において治験薬とは、被験薬及び対照薬をいう。この省令において製造販売後臨床試験薬とは、被験薬及び対照薬をいう。この省令において治験使用薬とは、被験薬並びに被験薬の有効性及び安全性の評価のために使用する薬物をいう。この省令において治験使用薬等とは、治験使用薬又は治験使用薬と成分が同一性を有すると認められる薬物をいう。この省令において製造販売後臨床試験使用薬とは、被験薬並びに被験薬の有効性及び安全性の評価のために使用する薬物をいう。この省令において製造販売後臨床試験使用薬等とは、製造販売後臨床試験使用薬又は製造販売後臨床試験使用薬と成分が同一性を有すると認められる薬物をいう。

In [ ]:
class LawLoader(object):
    """
    Prepare law data with e-Gov (https://www.e-gov.go.jp/) site.

    Args:
        category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)
    """

    def __init__(self, category=1):
        self.law_dict = self._get_law_dict(category=category)
        self.content_dict = {}

    @staticmethod
    def _get_xml(url):
        """
        Get XML data from e-Gov API.

        Args:
            url (str): key of the API

        Returns:
            xml.ElementTree: element tree of the XML data
        """
        r = requests.get(url)
        return ElementTree.fromstring(r.content.decode(encoding="utf-8"))

    def _get_law_dict(self, category):
        """
        Return dictionary of law names and numbers.

        Args:
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law names (keys) and numbers (values)
        """
        url = f"https://elaws.e-gov.go.jp/api/1/lawlists/{category}"
        root = self._get_xml(url)
        names = [e.text for e in root.iter() if e.tag == "LawName"]
        numbers = [e.text for e in root.iter() if e.tag == "LawNo"]
        return {name: num for (name, num) in zip(names, numbers)}

    def get_law_number(self, keyword, category=1):
        """
        Return the law number.
        This will be retrieved from e-Gov (https://www.e-gov.go.jp/)

        Args:
            keyword (str): keyword of the law name
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law name (key) and law number (value)
        """
        return {k: v for (k, v) in self.law_dict.items() if keyword in k}

    def get_raw(self, number):
        """
        Args:
            number (str): Number of the law, like '平成九年厚生省令第二十八号'

        Returns:
            raw (list[str]): raw contents of J-GCP
        """
        if number in self.content_dict:
            return self.content_dict[number]
        url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{number}"
        root = self._get_xml(url)
        contents = [e.text.strip() for e in root.iter() if e.text]
        raw = [t for t in contents if t]
        self.content_dict = {number: raw}
        return raw

    @staticmethod
    def pre_process(raw):
        """
        Perform pre-processing on raw contents.

        Args:
            raw (list[str]): raw contents

        Returns:
            str: pre-processed string

        Notes:
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        contents = [s for s in raw if s.endswith("。")]
        string = "".join(contents)
        string = string.translate(str.maketrans({"「": "", "」": ""}))
        return re.sub("（[^（|^）]*）", "", string)

    def gcp(self):
        """
        Perform pre-processing on raw contents of J-GCP.

        Args:
            raw (list[str]): raw contents of J-GCP

        Returns:
            str: pre-processed string of J-GCP

        Notes:
            - Article 56 will be removed.
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        number_dict = self.get_law_number("医薬品の臨床試験")
        number = number_dict["医薬品の臨床試験の実施の基準に関する省令"]
        raw = self.get_raw(number)
        raw_without56 = raw[: raw.index("第五十六条")]
        return self.pre_process(raw_without56)

In [ ]:
# The Constitution of Japan
loader2 = LawLoader(category=2)
consti_number = loader2.get_law_number("日本国憲法")
print(consti_number)  # -> '昭和二十一年憲法'
consti_raw = loader2.get_raw("昭和二十一年憲法")
consti = loader2.pre_process(consti_raw)
# J-GCP：データ整形を含めてメソッドとして登録済
loader4 = LawLoader(category=4)
gcp = loader4.gcp()

{'日本国憲法': '昭和二十一年憲法', '昭和二十二年法律第七十二号（日本国憲法施行の際現に効力を有する命令の規定の効力等に関する法律）': '昭和二十二年法律第七十二号', '日本国憲法の改正手続に関する法律': '平成十九年法律第五十一号'}
